In [213]:
from datetime import datetime

In [214]:
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [215]:
sg = Subgrounds()
snapshot = sg.load_subgraph('https://hub.snapshot.org/graphql')

In [112]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [113]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=100,
  where=[
    snapshot.Proposal.space == 'olympusdao.eth',
    snapshot.Proposal.state == 'closed',
  ]
)

In [190]:
olympus_snapshots = sg.query_df([
  proposals.title,
  proposals.id,
  proposals.body
])
type(olympus_snapshots)

pandas.core.frame.DataFrame

In [191]:
proposals_choices = sg.query(proposals.choices)

In [192]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_a', 'option_b', 'option_c', 'option_d', 'option_e'])

In [193]:
olympus_governance_view = pd.concat([olympus_snapshots,proposals_choices], axis=1)

In [276]:
olympus_governance_view

,proposals_title,proposals_id,proposals_body,option_a,option_b,option_c,option_d,option_e
0,OIP-80: Tender Offer Framework,0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04...,### Proposal\n\nThe Olympus team seeks communi...,Approve the TFO,Do not approve the TFO,None,None,None
1,OIP-77: New Tier for Bug Bounty & Awarding of ...,0xf1474731033990ba04abd897e70c5387160fab81e23e...,Summary\n\nThis proposal is designed to add a ...,Change Bug Bounty Program,Do nothing,None,None,None
2,OIP-71 Goddess DAO Launch Liquidity Assistance...,0x88b17056b2677673df2731eec8431a3aadcdc7f8f348...,Summary\n\nAssist Goddess DAO with gOHM launch...,Approve the launch liquidity as,Do nothing,None,None,None
3,OIP-75: Olympus Compensation Process,0xec57e860aa083fdd2fda66d1787ef8f965bd2f75b239...,Please see the forum for full details and disc...,For: adopt standard approach,Against: continue as-is,None,None,None
4,OIP-74: Partnership with JonesDAO,0x3623c7cd9452d348bc7bdbeeb372dc135db6e38e9612...,Summary\n\nFormalize a partnership to integrat...,Approve Partnership,Do nothing,None,None,None
...,...,...,...,...,...,...,...,...
95,any airdrop?,0xe912fb26a4e50b93cac27c09d54b716d16f7f81c769a...,any airdrop?,yes,yes,None,None,None
96,hey what coin is this,0x47d1cd5c6aa55b6b97df6a92066aeb1c958e57cd4a3e...,send me eth please,1,2,None,None,None
97,fuck you,0x47b42407c756b3cdf244a4c406f2153ba8615fc920a4...,what the fuck,fuck,mom,None,None,None
98,AIRDROP,0xcf049d7e9900e80bbfa74b568cdf0c6ab7408706c8a0...,We need AIRDROP!,Yes,No,None,None,None


In [208]:
votes_view = snapshot.Query.votes(
  orderBy='created',
  orderDirection='desc',
  first=100,
  where=[
    snapshot.Proposal.id == '0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04b958ebb7606abf31174b',
  ]
)

In [211]:
import requests

def run_query(query):
    request = requests.post('https://hub.snapshot.org/graphql'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))



    # Step 1 Create a query using graph API
query = """
{
  votes (
    first: 100
    skip: 0
    where: {
      space: "olympusdao.eth"
      proposal: "0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04b958ebb7606abf31174b"
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    voter
    created
    proposal {
      id
    }
    choice
    space {
      id
    }
  }
}

"""
results = run_query(query)

In [212]:
results

{'data': {'votes': [{'id': '0x82878f23fe56c8a63d8ee78aa3e1694a651b0fdf03a3b4e7ee9bc44c9a5f7c7f',
    'voter': '0x78B3367B0F547Cd16Bac453F788956AfcE92447B',
    'created': 1645049500,
    'proposal': {'id': '0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04b958ebb7606abf31174b'},
    'choice': 1,
    'space': {'id': 'olympusdao.eth'}},
   {'id': '0xd433b5f952ed9e3ec330fc2db2483074729202ec014730221daedd8adb2c687b',
    'voter': '0x3A9C0A0E46341B4D8a5ae5499b8ECcF4586FD512',
    'created': 1645049446,
    'proposal': {'id': '0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04b958ebb7606abf31174b'},
    'choice': 1,
    'space': {'id': 'olympusdao.eth'}},
   {'id': '0x043229dfdb2b53c61832b2b4658133c102970287527b07b2ea03cf50dc592772',
    'voter': '0xf7Db1796EE8603f3a5De43793581E038027fB9fb',
    'created': 1645047362,
    'proposal': {'id': '0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04b958ebb7606abf31174b'},
    'choice': 1,
    'space': {'id': 'olympusdao.eth'}},
   {'id': '0x41cfad408f0e740d3d7

In [228]:
votes_snapshot = sg.load_subgraph('https://hub.snapshot.org/graphql')

In [231]:
vote_tracker = votes_snapshot.Query.votes
#vote_tracker_votes = vote_tracker.votes
vote_tracker_id = vote_tracker.id
vote_tracker_voter = vote_tracker.voter
vote_tracker_created = vote_tracker.created
vote_tracker_choice = vote_tracker.choice

In [235]:
req1 = sg.mk_request([
vote_tracker_voter
])

In [236]:
print(req1.graphql)

query {
  votes {
    voter
  }
}


In [260]:
vote_tracker_2 = votes_snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=100,
    where=[
        votes_snapshot.Vote.proposal == '0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04b958ebb7606abf31174b'
    ]
)

In [261]:
req2 = sg.mk_request([
    vote_tracker_2.id,
    vote_tracker_2.voter,
    vote_tracker_2.created,
    vote_tracker_2.choice
])

In [262]:
print(req2.graphql)

query {
  x4d58b104ccb9a1b4: votes(first: 100, where: {proposal: "0xfbad1f4af42f99e59e0ef216211f2ed8b1f3a324de04b958ebb7606abf31174b"}, orderBy: "created", orderDirection: desc) {
    id
    voter
    created
    choice
  }
}


In [263]:
sg.query(vote_tracker_2)

[{'id': '0x82878f23fe56c8a63d8ee78aa3e1694a651b0fdf03a3b4e7ee9bc44c9a5f7c7f'},
 {'id': '0xd433b5f952ed9e3ec330fc2db2483074729202ec014730221daedd8adb2c687b'},
 {'id': '0x043229dfdb2b53c61832b2b4658133c102970287527b07b2ea03cf50dc592772'},
 {'id': '0x41cfad408f0e740d3d7212cf829765fe1dba94df860fc2f05f808e51556959f8'},
 {'id': '0x263c8568340308078912b543f145c56cbbcb1663f74ea94b32e8e7dfb926d8bb'},
 {'id': '0x677aba0e90dbeb334da2bcc45ec1ad32313d12252d23a03520e9b82ee5552d34'},
 {'id': '0xe4b3a249cbaa601efb45e22c3da6f633f65ac992d814267d673024f203994424'},
 {'id': '0xf2852f3c6951f2b097f6dc13a756466938ca7e3a631e9a865d55968c3b6552f1'},
 {'id': '0x5c8365f88b1b6e97deca36fb7571bb00586a97375b580b6317a8570ae53428b3'},
 {'id': '0x2b19f678ee1f081e06304b1674c9b2fab65639c808cac7a1799981f43e820c6f'},
 {'id': '0xa87517cf52c1d9a5502198395dc0b8b89e965162b08e9b772575d581f40d0342'},
 {'id': '0xc854a9b3075e70b6bf63af2e7ffd699468ce036c50f689aa5a06bf04b86eab70'},
 {'id': '0xed2850eda6e26c484e2ef5a649f6c2ec8c6eedb8c

In [267]:
sg.query_df([
    vote_tracker_2.id,
    vote_tracker_2.voter,
    vote_tracker_2.created,
    vote_tracker_2.choice
])

,votes_id,votes_voter,votes_created,votes_choice
0,0x82878f23fe56c8a63d8ee78aa3e1694a651b0fdf03a3...,0x78B3367B0F547Cd16Bac453F788956AfcE92447B,1645049500,1
1,0xd433b5f952ed9e3ec330fc2db2483074729202ec0147...,0x3A9C0A0E46341B4D8a5ae5499b8ECcF4586FD512,1645049446,1
2,0x043229dfdb2b53c61832b2b4658133c102970287527b...,0xf7Db1796EE8603f3a5De43793581E038027fB9fb,1645047362,1
3,0x41cfad408f0e740d3d7212cf829765fe1dba94df860f...,0xC8C6BeD3c421d3B5ddd0627C7d3AD1F95EaB2D67,1645042434,1
4,0x263c8568340308078912b543f145c56cbbcb1663f74e...,0x852a7C8655daa996E7E4F0E68e1b4D0950c416eb,1645042415,1
...,...,...,...,...
95,0x76b28aaa3dd12ba13925e01e5d4abe354dfbf986d54f...,0xCA7345ebd16d8d4D5aC6C38FaE99154fE1ecA90B,1644936720,1
96,0x096e9ff86e047b1b1f34d7138c7eae163d36c2704528...,0xF59C17EB3b068d5B44E1F8348A5aC56e66347f48,1644934061,1
97,0xa77ff182888d517ebd4848bbae29c84f83b71771d10e...,0x4541C852aDdC826ea6D4A1fF6d55e855490FaCc7,1644933811,1
98,0x3a41071329fe69ca1dbb58cb8f1d80748d374bdd27f5...,0x9C252F5e4E94287A19B009Cc7bD98671De7cC7F8,1644933702,1
